In [2]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v8.0 - REAL CHAIN INJECTION
Construimos sobre un bloque REAL de la cadena
"""

import hashlib
import struct
import socket
import time
import random
import select
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass
from collections import defaultdict
from datetime import datetime

@dataclass
class BlockHeader:
    version: int
    prev_hash: bytes
    merkle_root: bytes
    timestamp: int
    bits: int
    nonce: int

    def serialize(self) -> bytes:
        return (struct.pack('<I', self.version) + self.prev_hash +
                self.merkle_root + struct.pack('<I', self.timestamp) +
                struct.pack('<I', self.bits) + struct.pack('<I', self.nonce))

    def hash(self) -> bytes:
        return hashlib.sha256(hashlib.sha256(self.serialize()).digest()).digest()

    def hash_hex(self) -> str:
        return self.hash()[::-1].hex()

@dataclass
class Block:
    header: BlockHeader
    coinbase_script: bytes = b''

    def serialize(self) -> bytes:
        data = self.header.serialize()
        data += b'\x01'
        data += self._coinbase()
        return data

    def _coinbase(self) -> bytes:
        tx = struct.pack('<I', 1)
        tx += b'\x01' + bytes(32) + struct.pack('<I', 0xffffffff)
        script = self.coinbase_script or b'\x04KAORU'
        tx += bytes([len(script)]) + script + struct.pack('<I', 0xffffffff)
        tx += b'\x01' + struct.pack('<Q', 50 * 10**8)
        tx += b'\x19\x76\xa9\x14' + bytes(20) + b'\x88\xac'
        tx += struct.pack('<I', 0)
        return tx


class KaoruBridgeV8:
    """
    v8.0 - REAL CHAIN INJECTION

    La clave: construir nuestro bloque sobre un bloque REAL
    que los nodos ya conocen.
    """

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_BLOCK = 2
    MSG_HEADERS = 0x40000002

    # Bloque reciente conocido (lo obtendremos de un nodo)
    known_tip_hash: Optional[bytes] = None
    known_tip_height: int = 0

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.our_blocks: List[Block] = []
        self.stats = defaultdict(int)
        self.received_headers: List[bytes] = []

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    def log(self, msg: str):
        t = datetime.now().strftime("%H:%M:%S.%f")[:-3]
        print(f"      [{t}] {msg}")

    # ============================================
    # MENSAJES
    # ============================================

    def msg(self, cmd: str, payload: bytes) -> bytes:
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip: str) -> bytes:
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x10/KaoruBridge:8.0/'
        p += struct.pack('<i', 900000)  # Decimos que estamos en altura alta
        p += b'\x01'
        return p

    def getheaders_payload(self, known_hashes: List[bytes] = None) -> bytes:
        """Pide headers desde hashes conocidos."""
        p = struct.pack('<I', 70016)  # version

        if known_hashes:
            p += bytes([len(known_hashes)])
            for h in known_hashes:
                p += h
        else:
            # Pedir desde el tip
            p += b'\x01'
            p += bytes(32)  # Hash cero = desde genesis

        p += bytes(32)  # stop hash
        return p

    def varint(self, n: int) -> bytes:
        if n < 0xfd:
            return bytes([n])
        elif n <= 0xffff:
            return b'\xfd' + struct.pack('<H', n)
        else:
            return b'\xfe' + struct.pack('<I', n)

    def read_varint(self, data: bytes, offset: int) -> Tuple[int, int]:
        first = data[offset]
        if first < 0xfd:
            return first, offset + 1
        elif first == 0xfd:
            return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
        elif first == 0xfe:
            return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5
        else:
            return struct.unpack('<Q', data[offset+1:offset+9])[0], offset + 9

    def parse_msgs(self, data: bytes) -> List[Tuple[str, bytes]]:
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def recv_all(self, sock: socket.socket, timeout: float = 3.0) -> bytes:
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(65536)
                    if chunk:
                        data += chunk
                    else:
                        break
            except:
                break
        sock.setblocking(True)
        return data

    # ============================================
    # FASE 1: Obtener el TIP real de la cadena
    # ============================================

    def get_chain_tip(self, ip: str) -> Optional[Tuple[bytes, int]]:
        """Conecta a un nodo y obtiene el tip actual de la cadena."""

        self.log(f"🔍 Obteniendo chain tip de {ip}...")

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, 8333))

            # Handshake
            sock.send(self.msg('version', self.version_payload(ip)))
            time.sleep(0.5)

            data = self.recv_all(sock, 3.0)

            for cmd, payload in self.parse_msgs(data):
                if cmd == 'version':
                    # Obtener altura del peer
                    if len(payload) >= 85:
                        height = struct.unpack('<i', payload[81:85])[0]
                        self.log(f"   Peer altura: {height}")

                    sock.send(self.msg('verack', b''))

            time.sleep(0.3)

            # Pedir headers más recientes
            # Usamos el genesis como punto de partida conocido
            genesis_hash = bytes.fromhex("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f")[::-1]

            sock.send(self.msg('getheaders', self.getheaders_payload([genesis_hash])))
            self.log("   Pidiendo headers...")

            time.sleep(1.0)
            data = self.recv_all(sock, 5.0)

            best_header = None
            best_height = 0

            for cmd, payload in self.parse_msgs(data):
                if cmd == 'headers':
                    count, offset = self.read_varint(payload, 0)
                    self.log(f"   Recibidos {count} headers")

                    if count > 0:
                        # Guardar el último header (más reciente que nos dieron)
                        for i in range(count):
                            if offset + 81 > len(payload):
                                break
                            header_data = payload[offset:offset+80]
                            self.received_headers.append(header_data)
                            offset += 81  # 80 bytes header + 1 byte tx count

                        # El último es el más alto que conocemos
                        if self.received_headers:
                            best_header = self.received_headers[-1]
                            best_height = count  # Aproximado

                            header_hash = self.double_sha256(best_header)[::-1].hex()
                            self.log(f"   Mejor header: {header_hash[:32]}...")

                elif cmd == 'inv':
                    # También pueden enviarnos INV con bloques nuevos
                    inv_count = payload[0] if payload else 0
                    self.log(f"   INV recibido: {inv_count} items")

            sock.close()

            if best_header:
                return best_header, best_height

            return None

        except Exception as e:
            self.log(f"   ❌ Error: {e}")
            return None

    # ============================================
    # FASE 2: Construir bloque sobre chain real
    # ============================================

    def create_block_on_real_chain(self, prev_header: bytes, message: str = "KAORU") -> Block:
        """Crea un bloque que continúa la cadena real."""

        prev_hash = self.double_sha256(prev_header)

        # Script con nuestro mensaje
        script = message.encode()[:100]
        merkle = self.double_sha256(script)

        # Extraer bits del header anterior para mantener dificultad similar
        prev_bits = struct.unpack('<I', prev_header[72:76])[0]

        header = BlockHeader(
            version=0x20000000,
            prev_hash=prev_hash,
            merkle_root=merkle,
            timestamp=int(time.time()),
            bits=prev_bits,  # Misma dificultad que el anterior
            nonce=0
        )

        return Block(header=header, coinbase_script=script)

    def mine_block(self, block: Block, target_zeros: int = 4, max_nonce: int = 50_000_000) -> bool:
        """Mina el bloque."""
        target = "0" * target_zeros
        start = time.time()

        for nonce in range(max_nonce):
            block.header.nonce = nonce
            h = block.header.hash_hex()

            if h.startswith(target):
                elapsed = time.time() - start
                self.log(f"   ✅ Minado! Hash: {h[:32]}...")
                self.log(f"      Nonce: {nonce:,} | Tiempo: {elapsed:.2f}s")
                return True

            if nonce % 1_000_000 == 0 and nonce > 0:
                self.log(f"      ... {nonce:,} hashes")

        return False

    # ============================================
    # FASE 3: Propagar el bloque
    # ============================================

    def propagate_block(self, ip: str, block: Block, duration: int = 45) -> Dict:
        """Propaga nuestro bloque a un nodo."""

        result = {
            'ip': ip,
            'connected': False,
            'handshake': False,
            'headers_sent': 0,
            'blocks_sent': 0,
            'getdata_received': False,
            'inv_sent': False,
            'events': []
        }

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, 8333))
            result['connected'] = True
            start_time = time.time()

            self.log("→ Conectado!")

            # Handshake
            sock.send(self.msg('version', self.version_payload(ip)))

            time.sleep(0.5)

            inv_sent = False

            while time.time() - start_time < duration:
                data = self.recv_all(sock, 2.0)

                if not data:
                    time.sleep(0.2)
                    continue

                for cmd, payload in self.parse_msgs(data):
                    self.stats[f'recv_{cmd}'] += 1

                    if cmd == 'version':
                        try:
                            ua_len = payload[80]
                            peer_agent = payload[81:81+ua_len].decode('ascii', errors='ignore')
                            self.log(f"← VERSION ({peer_agent[:30]})")
                        except:
                            self.log("← VERSION")

                        sock.send(self.msg('verack', b''))
                        self.stats['sent_verack'] += 1

                    elif cmd == 'verack':
                        self.log("← VERACK")
                        result['handshake'] = True

                        if not inv_sent:
                            time.sleep(0.3)
                            # Anunciar nuestro bloque
                            inv_payload = b'\x01' + struct.pack('<I', self.MSG_BLOCK) + block.header.hash()
                            sock.send(self.msg('inv', inv_payload))
                            self.stats['sent_inv'] += 1
                            result['inv_sent'] = True
                            inv_sent = True

                            self.log(f"→ INV (anunciando bloque {block.header.hash_hex()[:24]}...)")
                            result['events'].append("→ INV")

                    elif cmd == 'ping':
                        nonce = payload[:8] if len(payload) >= 8 else payload
                        sock.send(self.msg('pong', nonce))
                        self.stats['sent_pong'] += 1
                        self.log(f"← PING → PONG")

                    elif cmd == 'sendcmpct':
                        sock.send(self.msg('sendcmpct', b'\x00' + struct.pack('<Q', 2)))
                        self.stats['sent_sendcmpct'] += 1

                    elif cmd == 'getheaders':
                        self.log("← GETHEADERS ⭐")
                        result['events'].append("← GETHEADERS")

                        # Enviar nuestro header
                        headers_payload = b'\x01' + block.header.serialize() + b'\x00'
                        sock.send(self.msg('headers', headers_payload))
                        self.stats['sent_headers'] += 1
                        result['headers_sent'] += 1

                        self.log(f"→ HEADERS (1 header enviado)")
                        result['events'].append("→ HEADERS")

                    elif cmd == 'getdata':
                        self.log("← GETDATA ⭐⭐⭐ ¡¡NOS PIDEN EL BLOQUE!!")
                        result['getdata_received'] = True
                        result['events'].append("← GETDATA ⭐⭐⭐")

                        # Parsear qué nos piden
                        count, offset = self.read_varint(payload, 0)

                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            # ¿Es nuestro bloque?
                            if inv_hash == block.header.hash():
                                self.log("→ BLOCK 📦📦📦 ¡¡ENVIANDO BLOQUE COMPLETO!!")

                                sock.send(self.msg('block', block.serialize()))
                                self.stats['sent_block'] += 1
                                result['blocks_sent'] += 1
                                result['events'].append("→ BLOCK 📦")

                    elif cmd == 'reject':
                        # Parsear el reject
                        try:
                            msg_len = payload[0]
                            msg = payload[1:1+msg_len].decode()
                            code = payload[1+msg_len]
                            reason_len = payload[2+msg_len]
                            reason = payload[3+msg_len:3+msg_len+reason_len].decode()
                            self.log(f"← REJECT ❌ {msg}: {reason} (code {code})")
                        except:
                            self.log(f"← REJECT ❌")
                        result['events'].append("← REJECT")

                    elif cmd == 'inv':
                        count = payload[0] if payload else 0
                        self.log(f"← INV ({count} items)")

                    elif cmd == 'headers':
                        count, _ = self.read_varint(payload, 0) if payload else (0, 0)
                        self.log(f"← HEADERS ({count})")

                    elif cmd == 'feefilter':
                        pass

                    elif cmd == 'wtxidrelay':
                        sock.send(self.msg('wtxidrelay', b''))

                    elif cmd == 'sendaddrv2':
                        pass

                    else:
                        self.log(f"← {cmd.upper()}")

            result['duration'] = time.time() - start_time
            sock.close()

        except Exception as e:
            result['error'] = str(e)
            self.log(f"❌ Error: {e}")

        return result

    def discover_nodes(self) -> List[str]:
        nodes = []
        for seed in ['seed.bitcoin.sipa.be', 'dnsseed.bluematt.me', 'seed.bitcoinstats.com']:
            try:
                nodes.extend(socket.gethostbyname_ex(seed)[2][:8])
            except:
                pass
        random.shuffle(nodes)
        return nodes

    # ============================================
    # EJECUCIÓN PRINCIPAL
    # ============================================

    def execute(self, num_peers: int = 3, session_duration: int = 45):
        print(f"""
╔════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                    ║
║   ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ █████╗     ██████╗       ║
║   ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║██╔══██╗   ██╔═████╗      ║
║   █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚█████╔╝   ██║██╔██║      ║
║   ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝██╔══██╗   ████╔╝██║      ║
║   ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝ ╚█████╔╝██╗╚██████╔╝      ║
║   ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚════╝ ╚═╝ ╚═════╝       ║
║                                                                                    ║
║                          REAL CHAIN INJECTION                                      ║
║                  "Construyendo sobre la cadena REAL"                               ║
║                                                                                    ║
╚════════════════════════════════════════════════════════════════════════════════════╝
        """)

        # FASE 1: Descubrir nodos
        print("=" * 90)
        print("[FASE 1] 🌐 DESCUBRIENDO NODOS")
        print("=" * 90)

        nodes = self.discover_nodes()
        print(f"\n   Encontrados: {len(nodes)} nodos")

        # FASE 2: Obtener chain tip real
        print("\n" + "=" * 90)
        print("[FASE 2] 🔗 OBTENIENDO CHAIN TIP REAL")
        print("=" * 90 + "\n")

        chain_tip = None
        for node_ip in nodes[:5]:
            result = self.get_chain_tip(node_ip)
            if result:
                chain_tip = result
                break

        if not chain_tip:
            print("\n   ❌ No se pudo obtener el chain tip")
            print("   Usando genesis block como fallback...")
            # Usar genesis como prev_hash
            genesis = bytes.fromhex("0100000000000000000000000000000000000000000000000000000000000000000000003ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a29ab5f49ffff001d1dac2b7c")
            chain_tip = (genesis, 0)

        tip_header, tip_height = chain_tip
        tip_hash = self.double_sha256(tip_header)[::-1].hex()

        print(f"\n   📋 Chain tip obtenido:")
        print(f"      Hash: {tip_hash[:48]}...")
        print(f"      Headers guardados: {len(self.received_headers)}")

        # FASE 3: Crear y minar bloque sobre chain real
        print("\n" + "=" * 90)
        print("[FASE 3] ⛏️  CREANDO BLOQUE SOBRE CHAIN REAL")
        print("=" * 90 + "\n")

        block = self.create_block_on_real_chain(tip_header, "KAORU_BRIDGE_v8_INJECTION")

        print(f"   Bloque creado:")
        print(f"      prev_hash: {block.header.prev_hash[::-1].hex()[:48]}...")
        print(f"      merkle:    {block.header.merkle_root[::-1].hex()[:48]}...")
        print(f"\n   ⛏️ Minando (esto puede tomar un momento)...")

        if self.mine_block(block, target_zeros=4):
            self.our_blocks.append(block)
            print(f"\n   ✅ Bloque minado!")
            print(f"      Hash: {block.header.hash_hex()}")
        else:
            print(f"\n   ❌ No se pudo minar")
            return

        # FASE 4: Propagar
        print("\n" + "=" * 90)
        print(f"[FASE 4] 📡 PROPAGANDO BLOQUE ({session_duration}s por peer)")
        print("=" * 90)

        results = []

        for ip in nodes[:num_peers]:
            print(f"\n   ╔{'═'*80}╗")
            print(f"   ║  📡 {ip:72} ║")
            print(f"   ╠{'═'*80}╣")

            result = self.propagate_block(ip, block, session_duration)
            results.append(result)

            print(f"   ╠{'═'*80}╣")
            print(f"   ║  📤 Headers enviados: {result['headers_sent']}                                                      ║")
            print(f"   ║  📦 Bloques enviados: {result['blocks_sent']}                                                      ║")
            print(f"   ║  🎯 GETDATA recibido: {'✅ SÍ!!' if result['getdata_received'] else '❌'}                                                   ║")
            print(f"   ╚{'═'*80}╝")

        # Resultados
        print("\n" + "=" * 90)
        print("                              📊 RESULTADOS FINALES")
        print("=" * 90)

        total_headers = sum(r['headers_sent'] for r in results)
        total_blocks = sum(r['blocks_sent'] for r in results)
        getdata_count = sum(1 for r in results if r['getdata_received'])

        print(f"""
   ╔═══════════════════════════════════════════════════════════════════════════════════╗
   ║                              ESTADÍSTICAS                                         ║
   ╠═══════════════════════════════════════════════════════════════════════════════════╣
   ║                                                                                   ║
   ║   Chain tip usado:          {tip_hash[:32]}...     ║
   ║   Bloque creado:            {block.header.hash_hex()[:32]}...     ║
   ║   Peers contactados:        {num_peers:3d}                                                 ║
   ║   Headers enviados:         {total_headers:3d}                                                 ║
   ║   GETDATA recibidos:        {getdata_count:3d}                                                 ║
   ║   BLOQUES ENVIADOS:         {total_blocks:3d}                                                 ║
   ║                                                                                   ║
   ╚═══════════════════════════════════════════════════════════════════════════════════╝
        """)

        print("   📊 Mensajes:")
        for key in sorted(self.stats.keys()):
            d = "📤" if 'sent' in key else "📥"
            print(f"      {d} {key.replace('recv_','').replace('sent_',''):15s}: {self.stats[key]}")

        print(f"""
   ╔═══════════════════════════════════════════════════════════════════════════════════╗
   ║                                                                                   ║
   ║   🏆 ESTADO:                                                                      ║
   ║                                                                                   ║
   ║   {'✅' if len(self.received_headers) > 0 else '❌'} Chain tip real obtenido                                               ║
   ║   {'✅' if len(self.our_blocks) > 0 else '❌'} Bloque construido sobre chain real                                      ║
   ║   {'✅' if total_headers > 0 else '❌'} Headers propagados                                                      ║
   ║   {'✅' if getdata_count > 0 else '❌'} Nodos solicitaron GETDATA                                              ║
   ║   {'✅' if total_blocks > 0 else '❌'} Bloques completos enviados                                              ║
   ║                                                                                   ║
   ╚═══════════════════════════════════════════════════════════════════════════════════╝

   Estado: KAORU BRIDGE v8.0 COMPLETADO ✅
        """)

        return results


if __name__ == "__main__":
    bridge = KaoruBridgeV8()
    bridge.execute(num_peers=3, session_duration=45)


╔════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                    ║
║   ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ █████╗     ██████╗       ║
║   ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║██╔══██╗   ██╔═████╗      ║
║   █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚█████╔╝   ██║██╔██║      ║
║   ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝██╔══██╗   ████╔╝██║      ║
║   ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝ ╚█████╔╝██╗╚██████╔╝      ║
║   ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚════╝ ╚═╝ ╚═════╝       ║
║                                                                                    ║
║                          REAL CHAIN INJECTION                                      ║
║                  "Construyendo sobre la cadena REAL"                               ║
║                                         